In [89]:
# gives us globally optimal solution with simplified assumptions, TODO: will refactor and explore rolling horizons

from docplex.mp.model import Model

model = Model(name="bus_dispatch")

# PARAMETERS
num_trips = 5 # |N|
num_stops = 5 # |S|
original_dispatch_list = [4, 6, 8, 10, 12] #j \in {1, .., |N|}
prev_arrival_list = [1, 2, 4, 7, 11] # j = 0, s \in {2, .. , |S|} # MODIFIED
prev_dwell_list = [1, 1, 1, 1] #j = 0, s \in {1, .., |S|-1}

original_dispatch = {j: original_dispatch_list[j-1] for j in range(1, num_trips+1)}
prev_arrival = {s: prev_arrival_list[s-1] for s in range(1, num_stops+1)} # MODIFIED
prev_dwell = {s: prev_dwell_list[s-1] for s in range(1, num_stops)}

target_headway_lists = [ # j \in {1, .. , |N|} s \in {2, .., |S|}
    [1, 2, 2, 1],
    [1, 2, 2, 1],
    [1, 2, 2, 1],
    [1, 2, 2, 1],
    [1, 2, 2, 1]
]

target_headway = {(j,s): target_headway_lists[j-1][s-2] for j in range(1, num_trips+1) for s in range(2, num_stops+1)}

interstation_travel_lists = [ # j \in {1, .. , |N|} s \in {1, .., |S|-1}
    [1, 2, 3, 4],
    [1, 2, 3, 4],
    [1, 2, 3, 4],
    [1, 2, 3, 4],
    [1, 2, 3, 4]
]

interstation_travel = {(j,s): interstation_travel_lists[j-1][s-1] for j in range(1, num_trips+1) for s in range(1, num_stops)}

arrival_rate_list = [0.2, 0.2, 0.2, 0.2, 0.2]
arrival_rate = {s: arrival_rate_list[s-1] for s in range(1, num_stops+1)}
alighting_percentage_list = [0.1, 0.2, 0.3, 0.4, 0.5]
alighting_percentage = {s: alighting_percentage_list[s-1] for s in range(1, num_stops+1)}
boarding_duration = 1
alighting_duration = 2
weights = [0.1, 0.2, 0.3, 0.4]
bus_availability_list = [5, 5, 5, 5, 5]
bus_availability = {j: bus_availability_list[j-1] for j in range(1, num_trips+1)}
num_buses = 5
bus_capacity = 100
initial_passengers = [0, 0, 0, 0, 0]
max_allowed_deviation = 10

# DECISION VARIABLES
dispatch_offset = model.continuous_var_dict(range(1,num_trips+1), name="dispatch_offset")
headway = model.continuous_var_matrix(range(1,num_trips+1), range(1,num_stops+1), name="headway")
arrival = model.continuous_var_matrix(range(1,num_trips+1), range(1,num_stops+1), name="arrival")
dwell = model.continuous_var_matrix(range(1,num_trips+1), range(1,num_stops+1), name="dwell")
willing_board = model.continuous_var_matrix(range(1,num_trips+1), range(1,num_stops+1), name="willing_board")
busload = model.continuous_var_matrix(range(1,num_trips+1), range(1,num_stops+1), name="busload")
slack = model.continuous_var(name="meta_slack")

# CONSTRAINTS

# constraint 6
model.add_constraint(headway[1,2] ==
                     (original_dispatch[1] + dispatch_offset[1])
                     + interstation_travel[(1,1)]
                     - prev_arrival[2])

for s in range(3, num_stops+1):
    model.add_constraint(headway[1,s] ==
                         headway[1,s-1]
                         + (dwell[1,s-1] + interstation_travel[(1,s-1)])
                         - (prev_arrival[s] - prev_arrival[s-1]))
    
# constraint 7
for j in range(2, num_trips+1):
    model.add_constraint(headway[j,2] ==
                        ((original_dispatch[j] + dispatch_offset[j]) + interstation_travel[(j,1)])
                        - ((original_dispatch[j-1] + dispatch_offset[j-1]) + interstation_travel[(j-1,1)]))
    
    for s in range(3, num_stops+1):
        model.add_constraint(headway[j,s] ==
                            headway[j,s-1]
                            + (dwell[j,s-1] + interstation_travel[j,s-1])
                            - (dwell[j-1,s-1] + interstation_travel[j-1,s-1]))

# constraint 8
beta = 1 / (num_trips * sum(weights))
f_x = beta * sum(stop_weights[s] * sum((headway[j,s] - target_headway[(j,s)]) ** 2 for j in range(1, num_trips))
                     for s in range(2, num_stops))
model.add_constraint(beta > 0)

# constraint 23
for j in range(1, num_trips+1):
    model.add_constraint(original_dispatch[j] + dispatch_offset[j] >= bus_availability[j])
    
# constraint 26
for s in range(2, num_stops):
    model.add_constraint(dwell[1,s] ==
                        boarding_duration * willing_board[1,s]
                        + alighting_duration * alighting_percentage[s] * busload[1,s])
    
# constraint 27
    model.add_constraint(willing_board[1,s] ==
                        (1 + arrival_rate[s] * boarding_duration)
                        * arrival_rate[s]
                        * (headway[1,s] - prev_dwell[s]))
    
# constraint 28
    for j in range(2, num_trips+1):
        for s in range(2, num_stops):
            model.add_constraint(dwell[j,s] ==
                                boarding_duration * willing_board[j,s]
                                + alighting_duration * alighting_percentage[s] * busload[j,s])
            
            #constraint 29
            model.add_constraint(willing_board[j,s] ==
                        (1 + arrival_rate[s] * boarding_duration)
                        * arrival_rate[s]
                        * (headway[j,s] - dwell[j-1,s]))
            
# constraint 30
model.add_constraint(busload[1,2] ==
                    (1 + arrival_rate[1] * boarding_duration)
                    * arrival_rate[1]
                    * (original_dispatch[1] + dispatch_offset[1] - prev_arrival[1] - prev_dwell[1]))

# constraint 31
for s in range(3, num_stops+1):
    model.add_constraint(busload[1,s] ==
                        busload[1,s-1]
                        + willing_board[1,s-1]
                        - alighting_percentage[s-1] * busload[1,s-1])
    
# constraint 32
for j in range(2, num_trips+1):
    model.add_constraint(busload[j,2] ==
                    (1 + arrival_rate[1] * boarding_duration)
                    * arrival_rate[1]
                    * (original_dispatch[j] + dispatch_offset[j] - original_dispatch[j-1] - dispatch_offset[j-1]))

# constraint 33
for j in range(2, num_trips+1):
    for s in range(3, num_stops+1):
        model.add_constraint(busload[j,s] ==
                        busload[j,s-1]
                        + willing_board[j,s-1]
                        - alighting_percentage[s-1] * busload[j,s-1])
        
# additional constraints to implement soft constraint:
for j in range(1, num_trips+1):
    #essentially its a smooth way to do max(x[j] - max_allowed_deviation, 0)
    model.add_constraint(slack >= dispatch_offset[j] - max_allowed_deviation)
model.add_constraint(slack >= 0)

# OBJECTIVE FUNCTION
objective_function = f_x + 1000 * slack # soft constraint using bigM = 1000 in case of infeasibility
model.minimize(objective_function)

# Solve the model
model.solve()

# Output the results
for j in range(1, num_trips+1):
        print(f"Trip {j}: Dispatch offset = {dispatch_offset[j].solution_value:1f}\
              Time of Dispatch = {original_dispatch[j] + dispatch_offset[j].solution_value}")

print("Objective Function Value:", model.objective_value)

Trip 1: Dispatch offset = 1.000000              Time of Dispatch = 5.000000000069105
Trip 2: Dispatch offset = 2.511127              Time of Dispatch = 8.511127443182383
Trip 3: Dispatch offset = 2.852253              Time of Dispatch = 10.852253468341019
Trip 4: Dispatch offset = 2.729055              Time of Dispatch = 12.72905486464707
Trip 5: Dispatch offset = 9.591629              Time of Dispatch = 21.591629481990054
Objective Function Value: 4.328447687661658
